# Extracción de Bases de datos
En este script extraeremos las bases de datos de las 6 viviendas. Las muestras seran cambiadas en su frecuencia tomando todos los datos cada segundo.

Los medidores de dispositivo llenaran los valores faltantes tomando el ultimo valor de tiempo en el pasado antes del dato faltante.

El relleno solo se aplica a las medidas de los electrodomesticos cuya frecuencia de muestreo es cada 3S, los medidores centrales tienen una frecuencia de muestreo de 1S por lo tanto sus valores no son modificados. 

Cuando un electrodomesticos es alimentado por dos fases del sistema de medición estos se aglomeran en un grupos, por ejemplo para la casa 1:

MeterGroup(meters=

    ElecMeter(instance=3, building=1, dataset='REDD', appliances=[Appliance(type='electric oven', instance=1)])
    
    ElecMeter(instance=4, building=1, dataset='REDD', appliances=[Appliance(type='electric oven', instance=1)])
  )
  
En estos casos al cargar los datos se retorna una sola medida la cual es la suma de las dos registradas originalmente.

Hay un id por medidor denominada instance, sin embargo tambien hay otro id por dispositivo denominado tambien instance al interior de la lista appliances. Este ultimo sirve como id en el caso cuando hay mas de un electrodomestico con un mismo nombre 'label'. 

En el caso que dos o mas dispositivos tengan el mismo 'label' la selección de uno en especifico se hace de la siguiente manera:

house_1['sockets',2]

En este ejemplo se selecciona el segundo conector de la casa 1. 

In [1]:
#0. Cargar las librerias 
from __future__ import print_function, division
import time
import nilmtk

from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from six import iteritems

from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.legacy.disaggregate import CombinatorialOptimisation, FHMM
import nilmtk.utils

%matplotlib inline
#Constante para el tamño de las imagenes
rcParams['figure.figsize'] = (13, 6)


In [2]:
# Periodo de muestreo
load_kwargs = {'sample_period': 1}

In [3]:
# Cargar Base de datos redd
redd = DataSet('C:\\Users\\josel\\SmartMeter\\OrdenandoTodo\\redd.h5')

In [4]:
# Cargar datos de las diferentes casas
house_1 = redd.buildings[1].elec
house_2 = redd.buildings[2].elec
house_3 = redd.buildings[3].elec
house_4 = redd.buildings[4].elec
house_5 = redd.buildings[5].elec
house_6 = redd.buildings[6].elec

# HOUSE 1

In [110]:
house_1

MeterGroup(meters=
  ElecMeter(instance=1, building=1, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=2, building=1, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=5, building=1, dataset='REDD', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=6, building=1, dataset='REDD', appliances=[Appliance(type='dish washer', instance=1)])
  ElecMeter(instance=7, building=1, dataset='REDD', appliances=[Appliance(type='sockets', instance=1)])
  ElecMeter(instance=8, building=1, dataset='REDD', appliances=[Appliance(type='sockets', instance=2)])
  ElecMeter(instance=9, building=1, dataset='REDD', appliances=[Appliance(type='light', instance=1)])
  ElecMeter(instance=11, building=1, dataset='REDD', appliances=[Appliance(type='microwave', instance=1)])
  ElecMeter(instance=12, building=1, dataset='REDD', appliances=[Appliance(type='unknown', instance=1)])
  ElecMeter(instance=13, building=1, dataset='REDD', appliances=[Appliance(type='electric 

## Para House_1

In [272]:
# Cargar datos por dispositivo
main_1 = house_1.mains()[1].power_series(**load_kwargs)
main_1 = next(main_1)

main_2 = house_1.mains()[2].power_series(**load_kwargs)
main_2 = next(main_2)

fridge_1 = house_1['fridge']
fridge_1 = next(fridge_1.power_series(**load_kwargs))

dish_washer_1 = house_1['dish washer']
dish_washer_1 = next(dish_washer_1.power_series(**load_kwargs))

sockets_1 = house_1['sockets',1]
sockets_1 = next(sockets_1.power_series(**load_kwargs))

sockets_2 = house_1['sockets',2]
sockets_2 = next(sockets_2.power_series(**load_kwargs))

light_1 = house_1['light',1]
light_1 = next(light_1.power_series(**load_kwargs))

microwave_1 = house_1['microwave']
microwave_1 = next(microwave_1.power_series(**load_kwargs))

unknown_1 = house_1['unknown',1]
unknown_1 = next(unknown_1.power_series(**load_kwargs))

electric_space_heater_1 = house_1['electric space heater'] 
electric_space_heater_1 = next(electric_space_heater_1.power_series(**load_kwargs))

electric_stove_1 = house_1['electric stove']
electric_stove_1 = next(electric_stove_1.power_series(**load_kwargs))

sockets_3 = house_1['sockets',3]
sockets_3 = next(sockets_3.power_series(**load_kwargs))

sockets_4 = house_1['sockets',4]
sockets_4 = next(sockets_4.power_series(**load_kwargs))

light_2 = house_1['light',2]
light_2 = next(light_2.power_series(**load_kwargs))

light_3 = house_1['light',3]
light_3 = next(light_3.power_series(**load_kwargs))

unknown_2 = house_1['unknown',2]
unknown_2 = next(unknown_2.power_series(**load_kwargs))

electric_oven_1 = house_1['electric oven']
electric_oven_1 = next(electric_oven_1.power_series(**load_kwargs))

washer_dryer_1 = house_1['washer dryer']
washer_dryer_1 = next(washer_dryer_1.power_series(**load_kwargs))

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\nilmtk\elecmeter.py:446: UserWarning: The provided sample_period (1) is shorter than the meter's sample_period (3)
  sample_period, default_sample_period


Loading data for meter ElecMeterID(instance=4, building=1, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=20, building=1, dataset='REDD')     
Done loading data all meters for this chunk.


In [284]:
label_h1 = ['main_1', 'main_2','fridge_1', 'dish washer_1', 'sockets_1','sockets_2', 'light_1', 'microwave_1', 'unknown_1',
            'electric_space heater_1', 'electric_stove_1', 'sockets_3', 'sockets_4', 'light_2', 'light_3',
            'unknown_2', 'electric_oven_1', 'washer_dryer_1']

In [285]:
len(label_h1)

18

In [279]:
type(washer_dryer_1)

pandas.core.series.Series

In [282]:
fridge_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_3.csv', sep = r' ')

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [290]:
main_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_1.csv', sep = r' ')

main_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_2.csv', sep = r' ')

dish_washer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_4.csv', sep = r' ') 

sockets_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_5.csv', sep = r' ') 

sockets_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_6.csv', sep = r' ')

light_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_7.csv', sep = r' ') 

microwave_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_8.csv', sep = r' ') 

unknown_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_9.csv', sep = r' ') 

electric_space_heater_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_10.csv', sep = r' ') 

electric_stove_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_11.csv', sep = r' ') 

sockets_3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_12.csv', sep = r' ')

sockets_4.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_13.csv', sep = r' ') 

light_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_14.csv', sep = r' ')

light_3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_15.csv', sep = r' ')

unknown_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_16.csv', sep = r' ')

electric_oven_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_17.csv', sep = r' ')

washer_dryer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\channel_18.csv', sep = r' ')

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an 

In [292]:
label_h1 = pd.DataFrame(label_h1)
label_h1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_1\\labels.csv', sep = r' ',header = None)

# HOUSE_2

In [312]:
house_2

MeterGroup(meters=
  ElecMeter(instance=1, building=2, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=2, building=2, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=3, building=2, dataset='REDD', appliances=[Appliance(type='sockets', instance=1)])
  ElecMeter(instance=4, building=2, dataset='REDD', appliances=[Appliance(type='light', instance=1)])
  ElecMeter(instance=5, building=2, dataset='REDD', appliances=[Appliance(type='electric stove', instance=1)])
  ElecMeter(instance=6, building=2, dataset='REDD', appliances=[Appliance(type='microwave', instance=1)])
  ElecMeter(instance=7, building=2, dataset='REDD', appliances=[Appliance(type='washer dryer', instance=1)])
  ElecMeter(instance=8, building=2, dataset='REDD', appliances=[Appliance(type='sockets', instance=2)])
  ElecMeter(instance=9, building=2, dataset='REDD', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=10, building=2, dataset='REDD', appliances=[Appliance(type='dis

In [313]:
label_h2 = ['main_1', 
            'main_2',
            'sockets_1',
            'light_1',
            'electric_stove_1',
            'microwave_1',
            'washer_dryer_1',
            'sockets_2',
            'fridge_1',
            'dish_washer_1',
            'waste_disposal_unit_1']
len(label_h2)

11

In [314]:
main_1 = house_2.mains()[1].power_series(**load_kwargs)
main_1 = next(main_1)
main_2 = house_2.mains()[2].power_series(**load_kwargs)
main_2 = next(main_2)

In [315]:
sockets_1 = next(house_2['sockets',1].power_series(**load_kwargs))

light_1 = next(house_2['light'].power_series(**load_kwargs))

electric_stove_1 = next(house_2['electric stove'].power_series(**load_kwargs))

microwave_1 = next(house_2['microwave'].power_series(**load_kwargs))

washer_dryer_1 = next(house_2['washer dryer'].power_series(**load_kwargs))

sockets_2 = next(house_2['sockets',2].power_series(**load_kwargs))

fridge_1 = next(house_2['fridge'].power_series(**load_kwargs))

dish_washer_1 = next(house_2['dish washer'].power_series(**load_kwargs))

waste_disposal_unit_1 = next(house_2['waste disposal unit'].power_series(**load_kwargs))

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\nilmtk\elecmeter.py:446: UserWarning: The provided sample_period (1) is shorter than the meter's sample_period (3)
  sample_period, default_sample_period


In [316]:
main_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_1.csv', sep = r' ')

main_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_2.csv', sep = r' ')

sockets_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_3.csv', sep = r' ')

light_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_4.csv', sep = r' ')

electric_stove_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_5.csv', sep = r' ')

microwave_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_6.csv', sep = r' ')

washer_dryer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_7.csv', sep = r' ')

sockets_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_8.csv', sep = r' ')

fridge_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_9.csv', sep = r' ')

dish_washer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_10.csv', sep = r' ')

waste_disposal_unit_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\channel_11.csv', sep = r' ')

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an 

In [317]:
label_h2 = pd.DataFrame(label_h2)
label_h2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_2\\labels.csv', sep = r' ',header = None)

# HOUSE 3

In [318]:
house_3

MeterGroup(meters=
  ElecMeter(instance=1, building=3, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=2, building=3, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=3, building=3, dataset='REDD', appliances=[Appliance(type='sockets', instance=1)])
  ElecMeter(instance=4, building=3, dataset='REDD', appliances=[Appliance(type='sockets', instance=2)])
  ElecMeter(instance=5, building=3, dataset='REDD', appliances=[Appliance(type='light', instance=1)])
  ElecMeter(instance=6, building=3, dataset='REDD', appliances=[Appliance(type='CE appliance', instance=1)])
  ElecMeter(instance=7, building=3, dataset='REDD', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=8, building=3, dataset='REDD', appliances=[Appliance(type='waste disposal unit', instance=1)])
  ElecMeter(instance=9, building=3, dataset='REDD', appliances=[Appliance(type='dish washer', instance=1)])
  ElecMeter(instance=10, building=3, dataset='REDD', appliances=[Appliance(ty

In [319]:
label_h3 = ['main_1','main_2',
'sockets_1',
'sockets_2',
'light_1',
'CE_appliance_1',
'fridge_1',
'waste_disposal_unit_1',
'dish_washer_1',
'electric_furnace_1',
'light_2',
'sockets_3',
'light_3',
'microwave_1',
'light_4',
'smoke_alarm_1',
'light_5',
'unknown_1',
'sockets_4',
'sockets_5',
'washer_dryer_1']
len(label_h3)

21

In [320]:
sockets_1 =  next(house_3['sockets',1].power_series(**load_kwargs))

sockets_2 = next(house_3['sockets',2].power_series(**load_kwargs))

light_1 = next(house_3['light',1].power_series(**load_kwargs))

CE_appliance_1 = next(house_3['CE appliance'].power_series(**load_kwargs))

fridge_1 = next(house_3['fridge'].power_series(**load_kwargs))

waste_disposal_unit_1 = next(house_3['waste disposal unit'].power_series(**load_kwargs))

dish_washer_1 = next(house_3['dish washer'].power_series(**load_kwargs))

electric_furnace_1 = next(house_3['electric furnace'].power_series(**load_kwargs))

light_2 = next(house_3['light',2].power_series(**load_kwargs))

sockets_3 = next(house_3['sockets',3].power_series(**load_kwargs))

light_3 = next(house_3['light',3].power_series(**load_kwargs))

microwave_1 = next(house_3['microwave'].power_series(**load_kwargs))

light_4 = next(house_3['light',4].power_series(**load_kwargs))

smoke_alarm_1 = next(house_3['smoke alarm'].power_series(**load_kwargs))

light_5 = next(house_3['light',5].power_series(**load_kwargs))

unknown_1 = next(house_3['unknown'].power_series(**load_kwargs))

sockets_4 = next(house_3['sockets',4].power_series(**load_kwargs))

sockets_5 = next(house_3['sockets',5].power_series(**load_kwargs))

washer_dryer_1 = next(house_3['washer dryer'].power_series(**load_kwargs))


C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\nilmtk\elecmeter.py:446: UserWarning: The provided sample_period (1) is shorter than the meter's sample_period (3)
  sample_period, default_sample_period


Loading data for meter ElecMeterID(instance=14, building=3, dataset='REDD')     
Done loading data all meters for this chunk.


In [321]:
main_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_1.csv', sep = r' ')

main_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_2.csv', sep = r' ')

sockets_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_3.csv', sep = r' ')

sockets_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_4.csv', sep = r' ')

light_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_5.csv', sep = r' ')

CE_appliance_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_6.csv', sep = r' ')

fridge_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_7.csv', sep = r' ')

waste_disposal_unit_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_8.csv', sep = r' ')

dish_washer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_9.csv', sep = r' ')

electric_furnace_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_10.csv', sep = r' ')

light_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_11.csv', sep = r' ')

sockets_3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_12.csv', sep = r' ')

light_3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_13.csv', sep = r' ')

microwave_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_14.csv', sep = r' ')

light_4.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_15.csv', sep = r' ')

smoke_alarm_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_16.csv', sep = r' ')

light_5.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_17.csv', sep = r' ')

unknown_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_18.csv', sep = r' ')

sockets_4.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_19.csv', sep = r' ')

sockets_5.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_20.csv', sep = r' ')

washer_dryer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\channel_21.csv', sep = r' ')

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an 

In [322]:
label_h3 = pd.DataFrame(label_h3)
label_h3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_3\\labels.csv', sep = r' ',header = None)

# HOUSE 4

In [ ]:
house_4

In [311]:
label_h4 = ['main_1',
'main_2',
'light_1',
'electric_furnace_1',
'sockets_1',
'sockets_2',
'washer_dryer_1', 
'electric_stove_1', 
'unknown_1',
'smoke_alarm_1',
'light_1',
'sockets_3',
'dish_washer_1',
'unknown_2',
'unknown_3',
'light_3', 
'light_4', 
'air_conditioner_2', 
'air_conditioner_1']
len(label_h4)

19

In [328]:
main_1 = next(house_4.mains()[1].power_series(**load_kwargs))

main_2 = next(house_4.mains()[2].power_series(**load_kwargs))

light_1 = next(house_4['light',1].power_series(**load_kwargs))

electric_furnace_1 = next(house_4['electric furnace'].power_series(**load_kwargs))

sockets_1 = next(house_4['sockets',1].power_series(**load_kwargs))

sockets_2 = next(house_4['sockets',2].power_series(**load_kwargs))

washer_dryer_1 =  next(house_4['washer dryer'].power_series(**load_kwargs))

electric_stove_1 =  next(house_4['electric stove'].power_series(**load_kwargs))

unknown_1 = next(house_4['unknown', 1].power_series(**load_kwargs))

smoke_alarm_1 = next(house_4['smoke alarm'].power_series(**load_kwargs))

light_1 = next(house_4['light',2].power_series(**load_kwargs))

sockets_3 = next(house_4['sockets',3].power_series(**load_kwargs))

dish_washer_1 = next(house_4['dish washer'].power_series(**load_kwargs))

unknown_2 = next(house_4['unknown',2].power_series(**load_kwargs))

unknown_3 = next(house_4['unknown',3].power_series(**load_kwargs))

light_3 = next(house_4['light', 3].power_series(**load_kwargs))

light_4 = next(house_4['light', 4].power_series(**load_kwargs)) 

air_conditioner_2 = next(house_4['air conditioner', 2].power_series(**load_kwargs))

air_conditioner_1 = next(house_4['air conditioner', 1].power_series(**load_kwargs))

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\nilmtk\elecmeter.py:446: UserWarning: The provided sample_period (1) is shorter than the meter's sample_period (3)
  sample_period, default_sample_period


Loading data for meter ElecMeterID(instance=10, building=4, dataset='REDD')     
Done loading data all meters for this chunk.


In [ ]:
main_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_1.csv', sep = r' ')

main_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_2.csv', sep = r' ')

light_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_3.csv', sep = r' ')

electric_furnace_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_4.csv', sep = r' ')

sockets_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_5.csv', sep = r' ')

sockets_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_6.csv', sep = r' ')

washer_dryer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_7.csv', sep = r' ')

electric_stove_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_8.csv', sep = r' ')

unknown_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_9.csv', sep = r' ')

smoke_alarm_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_10.csv', sep = r' ')

light_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_11.csv', sep = r' ')

sockets_3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_12.csv', sep = r' ')

dish_washer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_13.csv', sep = r' ')

unknown_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_14.csv', sep = r' ')

In [329]:
unknown_3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_15.csv', sep = r' ')

light_3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_16.csv', sep = r' ')

light_4.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_17.csv', sep = r' ')

air_conditioner_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_18.csv', sep = r' ')

air_conditioner_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\channel_19.csv', sep = r' ')

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an 

In [330]:
label_h4 = pd.DataFrame(label_h4)
label_h4.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_4\\labels.csv', sep = r' ',header = None)

# HOUSE 5

In [331]:
house_5

MeterGroup(meters=
  ElecMeter(instance=1, building=5, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=2, building=5, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=3, building=5, dataset='REDD', appliances=[Appliance(type='microwave', instance=1)])
  ElecMeter(instance=4, building=5, dataset='REDD', appliances=[Appliance(type='light', instance=1)])
  ElecMeter(instance=5, building=5, dataset='REDD', appliances=[Appliance(type='sockets', instance=1)])
  ElecMeter(instance=6, building=5, dataset='REDD', appliances=[Appliance(type='electric furnace', instance=1)])
  ElecMeter(instance=7, building=5, dataset='REDD', appliances=[Appliance(type='sockets', instance=2)])
  ElecMeter(instance=10, building=5, dataset='REDD', appliances=[Appliance(type='subpanel', instance=1)])
  ElecMeter(instance=11, building=5, dataset='REDD', appliances=[Appliance(type='subpanel', instance=2)])
  ElecMeter(instance=14, building=5, dataset='REDD', appliances=[Appliance(type='l

In [332]:
label_h5 = [
'main_1',
'main_2',
'microwave_1',
'light_1',  
'sockets_1', 
'electric_furnace_1',
'sockets_2',
'subpanel_1',
'subpanel_2',
'light_2',
'sockets_3',
'unknown_1',
'light_3',
'fridge_1',
'light_4',
'dish_washer_1',
'waste_disposal_unit_1',
'CE_appliance_1',
'light_5',
'sockets_4',
'sockets_5',
'sockets_6',
'washer_dryer_1', 
'electric_space_heater_1']
len(label_h5)

24

In [333]:
main_1 = next(house_5.mains()[1].power_series(**load_kwargs))
main_2 = next(house_5.mains()[2].power_series(**load_kwargs))

microwave_1 = next(house_5['microwave'].power_series(**load_kwargs))

light_1 = next(house_5['light', 1].power_series(**load_kwargs))

sockets_1 = next(house_5['sockets',1].power_series(**load_kwargs))

electric_furnace_1 = next(house_5['electric furnace'].power_series(**load_kwargs))

sockets_2 = next(house_5['sockets',2].power_series(**load_kwargs))

subpanel_1 =  next(house_5['subpanel', 1].power_series(**load_kwargs))

subpanel_2 =  next(house_5['subpanel', 2].power_series(**load_kwargs))

light_2 =  next(house_5['light', 2].power_series(**load_kwargs))

sockets_3 =  next(house_5['sockets', 3].power_series(**load_kwargs))

unknown_1 =  next(house_5['unknown'].power_series(**load_kwargs))

light_3 =  next(house_5['light', 3].power_series(**load_kwargs))

fridge_1 = next(house_5['fridge'].power_series(**load_kwargs)) 

light_4 = next(house_5['light', 4].power_series(**load_kwargs)) 

dish_washer_1 = next(house_5['dish washer'].power_series(**load_kwargs))

waste_disposal_unit_1 =  next(house_5['waste disposal unit'].power_series(**load_kwargs))

CE_appliance_1 =  next(house_5['CE appliance'].power_series(**load_kwargs))

light_5 = next(house_5['light', 5].power_series(**load_kwargs))

sockets_4 = next(house_5['sockets', 4].power_series(**load_kwargs))

sockets_5 = next(house_5['sockets', 5].power_series(**load_kwargs))

sockets_6 = next(house_5['sockets', 6].power_series(**load_kwargs))

washer_dryer_1 = next(house_5['washer dryer'].power_series(**load_kwargs)) 

electric_space_heater_1 = next(house_5['electric space heater'].power_series(**load_kwargs))

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\nilmtk\elecmeter.py:446: UserWarning: The provided sample_period (1) is shorter than the meter's sample_period (3)
  sample_period, default_sample_period


Loading data for meter ElecMeterID(instance=9, building=5, dataset='REDD')     
Done loading data all meters for this chunk.
Loading data for meter ElecMeterID(instance=13, building=5, dataset='REDD')     
Done loading data all meters for this chunk.


In [334]:
main_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_1.csv', sep = r' ')
main_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_2.csv', sep = r' ')
microwave_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_3.csv', sep = r' ')
light_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_4.csv', sep = r' ')
sockets_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_5.csv', sep = r' ')
electric_furnace_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_6.csv', sep = r' ')
sockets_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_7.csv', sep = r' ')
subpanel_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_8.csv', sep = r' ')
subpanel_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_9.csv', sep = r' ')
light_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_10.csv', sep = r' ')
sockets_3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_11.csv', sep = r' ')
unknown_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_12.csv', sep = r' ')
light_3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_13.csv', sep = r' ')
fridge_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_14.csv', sep = r' ')
light_4.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_15.csv', sep = r' ')
dish_washer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_16.csv', sep = r' ')
waste_disposal_unit_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_17.csv', sep = r' ')
CE_appliance_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_18.csv', sep = r' ')
light_5.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_19.csv', sep = r' ')
sockets_4.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_20.csv', sep = r' ')
sockets_5.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_21.csv', sep = r' ')
sockets_6.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_22.csv', sep = r' ')
washer_dryer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_23.csv', sep = r' ')
electric_space_heater_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\channel_24.csv', sep = r' ')

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel 

In [335]:
label_h5 = pd.DataFrame(label_h5)
label_h5.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_5\\labels.csv', sep = r' ',header = None)

# HOUSE 6

In [336]:
house_6

MeterGroup(meters=
  ElecMeter(instance=1, building=6, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=2, building=6, dataset='REDD', site_meter, appliances=[])
  ElecMeter(instance=3, building=6, dataset='REDD', appliances=[Appliance(type='sockets', instance=1)])
  ElecMeter(instance=4, building=6, dataset='REDD', appliances=[Appliance(type='washer dryer', instance=1)])
  ElecMeter(instance=5, building=6, dataset='REDD', appliances=[Appliance(type='electric stove', instance=1)])
  ElecMeter(instance=6, building=6, dataset='REDD', appliances=[Appliance(type='CE appliance', instance=1)])
  ElecMeter(instance=7, building=6, dataset='REDD', appliances=[Appliance(type='unknown', instance=1)])
  ElecMeter(instance=8, building=6, dataset='REDD', appliances=[Appliance(type='fridge', instance=1)])
  ElecMeter(instance=9, building=6, dataset='REDD', appliances=[Appliance(type='dish washer', instance=1)])
  ElecMeter(instance=10, building=6, dataset='REDD', appliances=[Appliance(

In [6]:
labels_h6 = [
'main_1',
'main_2',
'sockets_1',
'washer_dryer_1',
'electric_stove_1',
'CE_appliance_1',
'unknown_1',
'fridge_1',
'dish_washer_1', 
'sockets_2',
'sockets_3',
'electric_space_heater_1',
'sockets_4',
'light_1', 
'air_handling_unit_1',
'air_conditioner_1'
]
len(labels_h6)

16

In [338]:
main_1 = next(house_6.mains()[1].power_series(**load_kwargs))

main_2 = next(house_6.mains()[2].power_series(**load_kwargs))

sockets_1 = next(house_6['sockets', 1].power_series(**load_kwargs))

washer_dryer_1 = next(house_6['washer dryer'].power_series(**load_kwargs))

electric_stove_1 = next(house_6['electric stove'].power_series(**load_kwargs))

CE_appliance_1 = next(house_6['CE appliance'].power_series(**load_kwargs))

unknown_1 = next(house_6['unknown'].power_series(**load_kwargs))

fridge_1 = next(house_6['fridge'].power_series(**load_kwargs))

dish_washer_1 =  next(house_6['dish washer'].power_series(**load_kwargs))

sockets_2 = next(house_6['sockets', 2].power_series(**load_kwargs))

sockets_3 = next(house_6['sockets', 3].power_series(**load_kwargs))

electric_space_heater_1 = next(house_6['electric space heater'].power_series(**load_kwargs))

sockets_4 = next(house_6['sockets', 4].power_series(**load_kwargs))

light_1 = next(house_6['light'].power_series(**load_kwargs))

air_handling_unit_1 = next(house_6['air handling unit'].power_series(**load_kwargs))

air_conditioner_1 = next(house_6['air conditioner'].power_series(**load_kwargs))

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\nilmtk\elecmeter.py:446: UserWarning: The provided sample_period (1) is shorter than the meter's sample_period (3)
  sample_period, default_sample_period


Loading data for meter ElecMeterID(instance=17, building=6, dataset='REDD')     
Done loading data all meters for this chunk.


In [339]:
main_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_1.csv', sep = r' ')

main_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_2.csv', sep = r' ')

sockets_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_3.csv', sep = r' ')

washer_dryer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_4.csv', sep = r' ')

electric_stove_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_5.csv', sep = r' ')

CE_appliance_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_6.csv', sep = r' ')

unknown_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_7.csv', sep = r' ')

fridge_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_8.csv', sep = r' ')

dish_washer_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_9.csv', sep = r' ')

sockets_2.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_10.csv', sep = r' ')

sockets_3.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_11.csv', sep = r' ')

electric_space_heater_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_12.csv', sep = r' ')

sockets_4.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_13.csv', sep = r' ')

light_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_14.csv', sep = r' ')

air_handling_unit_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_15.csv', sep = r' ')

air_conditioner_1.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\channel_16.csv', sep = r' ')

C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\josel\Anaconda3\envs\nilmtk-env\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an 

In [7]:
labels_h6 = pd.DataFrame(labels_h6)
labels_h6.to_csv('C:\\Users\\josel\\nilmtk_work\\NILMTK_Repaso\\redd_low_frec\\house_6\\labels.csv', sep = r' ',header = None)